In [ ]:
import os
from google.adk.agents import Agent
from  main import retrieve_content
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents.callback_context import CallbackContext
os.environ["GOOGLE_API_KEY"] = ""
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
from google.adk.agents import LlmAgent

In [2]:

session_service = InMemorySessionService()

APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" 
initial_state = {"tool_result_list": []}
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state = initial_state
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

Session created: App='weather_tutorial_app', User='user_1', Session='session_001'


In [3]:
# instructions = """
# You are a specialized assistant for processing documents. Your ONLY task is to understand a user's request, extract the necessary information, and then call the `retrieve_content` tool.

# Follow these rules strictly:
# 1.  You will be given a GCS path and a user prompt.
# 2.  From the user's prompt, you MUST extract  a 'heading_number' (as a string) or a 'heading_title' (as a string).
# 3.  If both parameters are found, pass both of them, if any one is found then pass the other as None when you call the tool.The gcs path needs to passed in all cases strictly.
# 4.  **Error Condition**: If you cannot find ANY explicit heading number or heading title in the user's prompt, DO NOT call the tool. Instead, respond directly to the user with the exact message: "Error: Your request must include a specific heading number or title."
# 5. The tool will output a list containing two values in string format, you have to output in the given structure:
# ["Location_1" : list[0],["Location_1" : list[1]], here list is the output of the tool "retreival_agent"
# """

# # 2. Create the Agent instance
# agent = LlmAgent(
#     name = "retreival_agent",
#     model="gemini-2.5-pro",
#     tools=[retrieve_content],
#     instruction=instructions,
#     output_key = "locations" 
# )



In [4]:
from typing import Any, Dict
from google.adk.tools import ToolContext

async def print_locations(tool_context: ToolContext) -> Dict[Any, Any]:
    """
    Print the stored locations from state.
    Expects that state["locations"] is a list of GCS paths or empty strings.
    """
    print(f"  [Tool Call] print_locations triggered by {tool_context.agent_name}")

    # Access the agent's state
    state_dict = tool_context.state.to_dict()
    locations = state_dict.get("locations", [])

    if not isinstance(locations, list):
        return {"error": "locations is not a list in state."}
    if len(locations) == 0:
        return {"error": "No locations found."}

    # Filter out empty strings
    non_empty = [loc for loc in locations if loc]

    if len(non_empty) < 2:
        return {"error": "Not enough valid locations to print."}

    formatted = f'["Location_1" : {non_empty[0]}, "Location_2" : {non_empty[1]}]'
    print(f"Formatted Output: {formatted}")

    return {"formatted_locations": formatted}


In [5]:
instructions = """
You are a specialized assistant for processing documents. Your ONLY task is to understand a user's request, extract the necessary information, and then call the `retrieve_content` tool.

Rules:
1. You will be given a GCS path and a user prompt.
2. From the user's prompt, you MUST extract a 'heading_number' (string) or a 'heading_title' (string).
3. If both are found, pass both; if only one is found, set the other to None. Always pass the GCS path.
4. Error Condition: If no heading number or title is found, respond with: "Error: Your request must include a specific heading number or title."
5. The `retrieve_content` tool will return a list of two strings (locations). Store this in state as `locations`.
6. After calling `retrieve_content`, immediately call `print_locations` to display the output in this format:
   ["Location_1" : list[0], "Location_2" : list[1]]
"""

agent = LlmAgent(
    name="retrieval_agent",
    model="gemini-2.5-pro",
    tools=[retrieve_content, print_locations],
    instruction=instructions,
    output_key="locations"
)



In [6]:

runner = Runner(
    agent=agent, 
    app_name=APP_NAME,  
    session_service=session_service 
)



In [7]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [8]:


async def run_conversation():
    await call_agent_async(
        "GCS File Location : gs://genai_ex_documents/main_docs/g20_pub.pdf , User Prompt:Give me the test cases corresponding to section 5 and 'Progress Control'",
        runner=runner,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

await run_conversation()



>>> User Query: GCS File Location : gs://genai_ex_documents/main_docs/g20_pub.pdf , User Prompt:Give me the test cases corresponding to section 5 and 'Progress Control'


g20_pub
Extracting pages: [1, 2, 3]
File already exists. Saving instead as: gs://genai_ex_documents/content_pages/g20_pub_toc_copy11.pdf
Uploading extracted PDF to gs://genai_ex_documents/content_pages/g20_pub_toc_copy11.pdf
Calling API to process: gs://genai_ex_documents/content_pages/g20_pub_toc_copy11.pdf...


c:\Users\adini\anaconda3\Lib\site-packages\vertexai\generative_models\_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


4
Starting content population...
Detected headers/footers: {'APPLICATION SOFTWARE TESTING', 'GUIDELINES FOR', 'Nil'}

...Population complete!

--- Final Populated JSON ---
{
  "1": {
    "title": "PURPOSE",
    "content": "The major purpose of this document is to provide a set of application software testing \nguidelines, to ensure that computer systems are properly tested, so as to pursue reliable \nand quality computer systems. \n \n \n \n\n \nSCOPE \n________________________________________________________________________________ \n \n_______________________________________________________________________________",
    "subsections": {}
  },
  "2": {
    "title": "SCOPE",
    "content": "This document is to give a set of guidelines for reference by application project teams on \nthe planning and carrying out of testing activities for application software.  It is \nimportant that users of this document (i.e. application project teams) should not \ntreat these guidelines as mandatory 

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000154D1748C20>


Attempting to save JSON to: gs://genai_ex_documents/retreived_content_files/g20_pub_Progress Control_json_8.json
Successfully saved JSON to GCS at: gs://genai_ex_documents/retreived_content_files/g20_pub_Progress Control_json_8.json


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000154D1C360D0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x00000154D15A00B0>, 571375.8840818)])']
connector: <aiohttp.connector.TCPConnector object at 0x00000154D1C35D10>


  [Tool Call] print_locations triggered by retrieval_agent


<<< Agent Response: {"Location_1" : "gs://genai_ex_documents/retreived_content_files/g20_pub_5_json_10.json", "Location_2" : "gs://genai_ex_documents/retreived_content_files/g20_pub_Progress Control_json_8.json"}


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000154D1BBA0D0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x00000154D15A00B0>, 571391.5706028)])']
connector: <aiohttp.connector.TCPConnector object at 0x00000154D1BBA210>
